In [ ]:
# 구글 드라이브 접근 코드
from google.colab import drive
drive.mount('/content/drive')
My_path = '/content/drive/MyDrive/LG_Academy/AI_Code/'

## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
# 재현성을 위한 코드
random_seed = 777

tf.keras.utils.set_random_seed(random_seed)
# tf.random.normal([5], 0, 1)

## 데이터 불러오기

In [ ]:
DataLength = 2774
NoOfSensor = 3
NoOfData   = 180

DataLength, NoOfSensor

In [ ]:
NormalSet   = np.zeros([NoOfData, DataLength, NoOfSensor])
AbnormalSet = np.zeros([NoOfData, DataLength, NoOfSensor])

for i in range(NoOfData):
    tempNormalPath   = My_path + './SpotWeldingData/Normal_%d'  %(i+1)
    tempAbnormalPath = My_path + './SpotWeldingData/Abnormal_%d'%(i+1)

    tempNormal       = np.array(pd.read_csv(tempNormalPath  , header = None))
    tempAbnormal     = np.array(pd.read_csv(tempAbnormalPath, header = None))

    NormalSet[i, :, :]         = tempNormal
    AbnormalSet[i, :, :]      = tempAbnormal


NormalSet.shape, AbnormalSet.shape

In [ ]:
TrainData = np.concatenate([NormalSet[   :-30, :, :], AbnormalSet[   :-30, :, :]], axis=0)
ValidData = np.concatenate([NormalSet[-30:   , :, :], AbnormalSet[-30:   , :, :]], axis=0)

TrainData.shape, ValidData.shape

## hyperparameter 설정

In [ ]:
learningRate = 0.0001
NoOfFilter   = 32
Epoch        = 200
latent_dim   = 2

## Keras 기반 AutoEncoder 구조(Architecture) 설계 - 1DCNN

In [ ]:
def ConvAutoEncoder(input_data, latent_dim):
    keras.backend.clear_session() # 세션 지우기
    tf.keras.utils.set_random_seed(random_seed)

    Encoder = keras.Sequential(name = 'Encoder')
    Encoder.add(keras.layers.InputLayer(shape = (input_data.shape[1], input_data.shape[2]),
                                        name = 'Encoder_Input'))                                          # Encoder Input Layer
    Encoder.add(keras.layers.Conv1D(filters = NoOfFilter, kernel_size=5, strides=3, padding='same',
                                    activation=keras.activations.relu, name = 'Encoder_Conv1'))           # Encoder Conv. Layer 1
    Encoder.add(keras.layers.Conv1D(filters = NoOfFilter, kernel_size=5, strides=3, padding='same',
                                    activation=keras.activations.relu, name = 'Encoder_Conv2'))           # Encoder Conv. Layer 2
    Encoder.add(keras.layers.Conv1D(filters = NoOfFilter, kernel_size=5, strides=3, padding='same',
                                    activation=keras.activations.relu, name = 'Encoder_Conv3'))           # Encoder Conv. Layer 3
    Encoder.add(keras.layers.Conv1D(filters = NoOfFilter, kernel_size=5, strides=3, padding='same',
                                    activation=keras.activations.relu, name = 'Encoder_Conv4'))           # Encoder Conv. Layer 4
    Encoder.add(keras.layers.Flatten())
    Encoder.add(keras.layers.Dense(latent_dim,
                                   activation=keras.layers.LeakyReLU(), name = 'Encoder_Output'))         # Encoder Output Layer(Hidden)



    Decoder = keras.Sequential(name = 'Decoder')
    Decoder.add(keras.layers.InputLayer(shape = (latent_dim,),    name = 'Decoder_Input'))                # Decoder Input Layer
    Decoder.add(keras.layers.Dense(units = 34*16, activation='relu', name = 'Decoder_Hidden'))            # Decoder Hidden Layer
    Decoder.add(keras.layers.Reshape((34, 16)))                                                           # Reshape Layer
    Decoder.add(keras.layers.Conv1DTranspose(filters = NoOfFilter, kernel_size=5, strides=3, padding='same',
                                             activation=keras.activations.relu, name = 'Decoder_ConvT1')) # Decoder Conv.Transpose Layer 1
    Decoder.add(keras.layers.Conv1DTranspose(filters = NoOfFilter, kernel_size=4, strides=3,
                                             activation=keras.activations.relu, name = 'Decoder_ConvT2')) # Decoder Conv.Transpose Layer 2
    Decoder.add(keras.layers.Conv1DTranspose(filters = NoOfFilter, kernel_size=6, strides=3,
                                             activation=keras.activations.relu, name = 'Decoder_ConvT3')) # Decoder Conv.Transpose Layer 3
    Decoder.add(keras.layers.Conv1DTranspose(filters = input_data.shape[2], kernel_size=5, strides=3,
                                             name = 'Decoder_Ouput'))                                     # Decoder Output Layer(Conv.Transpose Layer)

    AutoEncoder = keras.Sequential([keras.layers.InputLayer(shape = (input_data.shape[1], input_data.shape[2])), Encoder, Decoder])

    AutoEncoder.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                        loss=keras.losses.MAE)
    return AutoEncoder

## AutoEncoder 구조 확인

In [ ]:
CAE_model = ConvAutoEncoder(TrainData, latent_dim)

CAE_model.summary()

In [ ]:
# Encoder 구조
encoder = CAE_model.layers[0]
encoder.summary()

In [ ]:
# Decoder 구조
decoder = CAE_model.layers[-1]
decoder.summary()

## Keras 기반 Autoencoder 학습 및 평가

In [ ]:
# 모델 학습
hist = CAE_model.fit(TrainData, TrainData, epochs=Epoch, verbose = 1)

In [ ]:
# 학습 과정 확인(Loss, Accuracy)
fig, loss_ax = plt.subplots()

loss_ax.plot(hist.history['loss'], label='train loss (mae)', color = "tab:red")
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

plt.show()

## Autoencoder model 예측 결과 확인

In [ ]:
Predict_result = CAE_model.predict(ValidData, verbose = 0)
Predict_result.shape

In [ ]:
# 예측 결과 그래프로 확인하기
# idx = 0
idx = tf.random.uniform([1], 0, Predict_result.shape[0], dtype = tf.int32)
idx = idx.numpy()[0]

plt.figure(figsize = (14, 8))
plt.subplot(3, 2, 1)
plt.plot(ValidData[idx, :, 0], c = 'r')
plt.grid(alpha = 0.8)
plt.title('Real')

plt.subplot(3, 2, 3)
plt.plot(ValidData[idx, :, 1], c = 'b')
plt.grid(alpha = 0.8)

plt.subplot(3, 2, 5)
plt.plot(ValidData[idx, :, 2], c = 'g')
plt.grid(alpha = 0.8)

plt.subplot(3, 2, 2)
plt.plot(Predict_result[idx, :, 0], c = 'r')
plt.grid(alpha = 0.8)
plt.title('Reconstruction')

plt.subplot(3, 2, 4)
plt.plot(Predict_result[idx, :, 1], c = 'b')
plt.grid(alpha = 0.8)

plt.subplot(3, 2, 6)
plt.plot(Predict_result[idx, :, 2], c = 'g')
plt.grid(alpha = 0.8)

plt.show()

## Latent Space 확인하기

In [ ]:
encoder = CAE_model.layers[0]
encoder.summary()

In [ ]:
latent_value = encoder.predict(ValidData, verbose = 0)
latent_value.shape

In [ ]:
# 데이터 길이 정의
NoOfValidData   = int(latent_value.shape[0]/2)
NoOfValidData

In [ ]:
# Latent Space 확인
plt.scatter(latent_value[:NoOfValidData,0], latent_value[:NoOfValidData,1], label = 'Normal'  , alpha = 0.8, c = 'b')
plt.scatter(latent_value[NoOfValidData:,0], latent_value[NoOfValidData:,1], label = 'Abnormal', alpha = 0.8, c = 'r')

plt.legend()
plt.grid(alpha = 0.4)
plt.show()

## Latent Space 기반 데이터 생성

In [ ]:
decoder = CAE_model.layers[1]
decoder.summary()

In [ ]:
temp_Normal      = np.array([125, -190]).reshape([1, 2])
temp_Normal_pred = decoder.predict(temp_Normal, verbose = 0)
temp_Normal_pred = temp_Normal_pred.reshape([2774, 3])

temp_Abnormal      = np.array([115, -180]).reshape([1, 2])
temp_Abnormal_pred = decoder.predict(temp_Abnormal, verbose = 0)
temp_Abnormal_pred = temp_Abnormal_pred.reshape([2774, 3])

# 시간열 정의
t = np.arange(0, 0.2167, 1/12800)

plt.figure(figsize = (14, 8))
plt.subplot(3, 2, 1)
plt.plot(t, temp_Normal_pred[:, 0], label = 'Acceleration', c = 'r')
plt.grid(alpha = 0.8)
plt.title('Normal_Generated')
plt.legend()

plt.subplot(3, 2, 3)
plt.plot(t, temp_Normal_pred[:, 1], label = 'voltage', c = 'b')
plt.grid(alpha = 0.8)
plt.legend()

plt.subplot(3, 2, 5)
plt.plot(t, temp_Normal_pred[:, 2], label = 'current', c = 'g')
plt.grid(alpha = 0.8)
plt.legend()

plt.subplot(3, 2, 2)
plt.plot(t, temp_Abnormal_pred[:, 0], label = 'Acceleration', c = 'r')
plt.grid(alpha = 0.8)
plt.title('Abnormal_Generated')
plt.legend()

plt.subplot(3, 2, 4)
plt.plot(t, temp_Abnormal_pred[:, 1], label = 'voltage', c = 'b')
plt.grid(alpha = 0.8)
plt.legend()

plt.subplot(3, 2, 6)
plt.plot(t, temp_Abnormal_pred[:, 2], label = 'current', c = 'g')
plt.grid(alpha = 0.8)
plt.legend()

plt.show()